<h1> Peer Assignment - Segmenting neighbourhoods of Toronto</h1>

In [14]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!python -m pip install geopy
#!conda config --add channels conda-forge
#!conda install geopy # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
#!python -m pip install folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\nachi\anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    certifi-2019.11.28         |           py37_0         148 KB  conda-forge
    conda-4.8.3                |   py37hc8dfbb8_1         3.1 MB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    python_abi-3.7             |          1_cp37m           4 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    --------------------------------------------------



==> WARNING: A newer version of conda exists. <==
  current version: 4.8.2
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base -c defaults conda




<h2>I am utilizing an API published on the Toronto data portal for the neighbourhood information instead of the Web scraping method. Not exactly following the guidelines in the submission intructions, but I believe the outcomes will be the similar.</h2> 

In [17]:
import pandas as pd
import requests

# Get the dataset metadata by passing package_id to the package_search endpoint
# For example, to retrieve the metadata for this dataset:

url = "https://ckan0.cf.opendata.inter.prod-toronto.ca/api/3/action/package_show"
params = { "id": "4def3f65-2a65-4a4f-83c4-b2a4aed72d46"}
package = requests.get(url, params = params).json()
print(package["result"])

# Get the data by passing the resource_id to the datastore_search endpoint
# See https://docs.ckan.org/en/latest/maintaining/datastore.html for detailed parameters options
# For example, to retrieve the data content for the first resource in the datastore:

for idx, resource in enumerate(package["result"]["resources"]):
    if resource["datastore_active"]:
        url = "https://ckan0.cf.opendata.inter.prod-toronto.ca/api/3/action/datastore_search"
        p = { "id": resource["id"] }
        data = requests.get(url, params = p).json()
        df = pd.DataFrame(data["result"]["records"])
        break
df.head()

{'license_title': 'Open Government Licence – Toronto', 'owner_unit': None, 'relationships_as_object': [], 'topics': 'Locations and mapping', 'owner_email': 'opendata@toronto.ca', 'excerpt': 'Boundaries of Toronto Neighbourhoods.', 'private': False, 'owner_division': 'Social Development, Finance & Administration', 'num_tags': 9, 'id': '4def3f65-2a65-4a4f-83c4-b2a4aed72d46', 'metadata_created': '2019-07-23T17:47:38.364157', 'refresh_rate': 'As available', 'title': 'Neighbourhoods', 'license_url': 'https://open.toronto.ca/open-data-license/', 'state': 'active', 'information_url': 'http://www.toronto.ca/demographics/neighbourhoods.htm', 'license_id': 'open-government-licence-toronto', 'type': 'dataset', 'resources': [{'cache_last_updated': None, 'package_id': '4def3f65-2a65-4a4f-83c4-b2a4aed72d46', 'datastore_active': True, 'id': 'a083c865-6d60-4d1d-b6c6-b0c8a85f9c15', 'size': None, 'format': 'GeoJSON', 'state': 'active', 'hash': '', 'description': '', 'is_preview': True, 'last_modified': 

,_id,AREA_ID,AREA_ATTR_ID,PARENT_AREA_ID,AREA_SHORT_CODE,AREA_LONG_CODE,AREA_NAME,AREA_DESC,X,Y,LONGITUDE,LATITUDE,OBJECTID,Shape__Area,Shape__Length,geometry
0,6301,25886861,25926662,49885,094,094,Wychwood (94),Wychwood (94),None,None,-79.425515,43.676919,16491505,3.217960e+06,7515.779658,"{""type"": ""Polygon"", ""coordinates"": [[[-79.4359..."
1,6302,25886820,25926663,49885,100,100,Yonge-Eglinton (100),Yonge-Eglinton (100),None,None,-79.403590,43.704689,16491521,3.160334e+06,7872.021074,"{""type"": ""Polygon"", ""coordinates"": [[[-79.4109..."
2,6303,25886834,25926664,49885,097,097,Yonge-St.Clair (97),Yonge-St.Clair (97),None,None,-79.397871,43.687859,16491537,2.222464e+06,8130.411276,"{""type"": ""Polygon"", ""coordinates"": [[[-79.3911..."
3,6304,25886593,25926665,49885,027,027,York University Heights (27),York University Heights (27),None,None,-79.488883,43.765736,16491553,2.541821e+07,25632.335242,"{""type"": ""Polygon"", ""coordinates"": [[[-79.5052..."
4,6305,25886688,25926666,49885,031,031,Yorkdale-Glen Park (31),Yorkdale-Glen Park (31),None,None,-79.457108,43.714672,16491569,1.156669e+07,13953.408098,"{""type"": ""Polygon"", ""coordinates"": [[[-79.4396..."


In [37]:
Toronto_DF = df[['AREA_LONG_CODE','AREA_NAME','LONGITUDE','LATITUDE']]
Toronto_DF['AREA_NAME']=Toronto_DF['AREA_NAME'].str.replace(r"\(.*\)","")
Toronto_DF

C:\Users\nachi\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,AREA_LONG_CODE,AREA_NAME,LONGITUDE,LATITUDE
0,094,Wychwood,-79.425515,43.676919
1,100,Yonge-Eglinton,-79.403590,43.704689
2,097,Yonge-St.Clair,-79.397871,43.687859
3,027,York University Heights,-79.488883,43.765736
4,031,Yorkdale-Glen Park,-79.457108,43.714672
5,114,Lambton Baby Point,-79.496045,43.657420
6,038,Lansing-Westgate,-79.424748,43.754271
7,105,Lawrence Park North,-79.403978,43.730060
8,103,Lawrence Park South,-79.406039,43.717212
9,056,Leaside-Bennington,-79.366072,43.703797


In [23]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [25]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, area_nm, area_code in zip(Toronto_DF['LATITUDE'], Toronto_DF['LONGITUDE'], Toronto_DF['AREA_NAME'], Toronto_DF['AREA_LONG_CODE']):
    label = '{}, {}'.format(area_nm, area_code)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [26]:
CLIENT_ID = 'XPOIF0PEXA5E2VEZUM1FGNDYZAONVFMHB4WTWRDEVB2TT4C0' # your Foursquare ID
CLIENT_SECRET = 'CJTJ45NF2ZL1VRNRHUIWXFNEKP45Q3QQSYOJPMSND4J1MQWL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XPOIF0PEXA5E2VEZUM1FGNDYZAONVFMHB4WTWRDEVB2TT4C0
CLIENT_SECRET:CJTJ45NF2ZL1VRNRHUIWXFNEKP45Q3QQSYOJPMSND4J1MQWL


<b> Explore more about the Annex neighbourhood in Toronto</b>

In [41]:
Toronto_DF.loc[82, 'AREA_NAME']

'Annex '

In [43]:
area_latitude = Toronto_DF.loc[82, 'LATITUDE'] # neighborhood latitude value
area_longitude = Toronto_DF.loc[82, 'LONGITUDE'] # neighborhood longitude value

area_name = Toronto_DF.loc[82, 'AREA_NAME'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(area_name, 
                                                               area_latitude, 
                                                               area_longitude))

Latitude and longitude values of Annex  are 43.6715854351, -79.4040006243.


In [44]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    area_latitude, 
    area_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=XPOIF0PEXA5E2VEZUM1FGNDYZAONVFMHB4WTWRDEVB2TT4C0&client_secret=CJTJ45NF2ZL1VRNRHUIWXFNEKP45Q3QQSYOJPMSND4J1MQWL&v=20180605&ll=43.6715854351,-79.4040006243&radius=500&limit=100'

In [45]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f0ca0843ff7134271e9ad6c'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'The Annex',
  'headerFullLocation': 'The Annex, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 24,
  'suggestedBounds': {'ne': {'lat': 43.6760854396, 'lng': -79.39779083260007},
   'sw': {'lat': 43.6670854306, 'lng': -79.41021041599993}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ae7bc29f964a52084ad21e3',
       'name': "Ezra's Pound",
       'location': {'address': '238 Dupont St',
        'crossStreet': 'Spadina Ave',
        'lat': 43.67515283323029,
        'lng': -79.40585846415303,
        'labeledLatLngs': [{'label': 'display',
 

In [46]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [47]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

C:\Users\nachi\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Ezra's Pound,Café,43.675153,-79.405858
1,Roti Cuisine of India,Indian Restaurant,43.674618,-79.408249
2,Madame Boeuf And Flea,Burger Joint,43.675240,-79.406620
3,Big Crow,BBQ Joint,43.675896,-79.403680
4,Jean Sibelius Square,Park,43.671426,-79.408831


In [48]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

24 venues were returned by Foursquare.


In [49]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [50]:
Toronto_venues = getNearbyVenues(names=Toronto_DF['AREA_NAME'],
                                   latitudes=Toronto_DF['LATITUDE'],
                                   longitudes=Toronto_DF['LONGITUDE']
                                  )

Wychwood 
Yonge-Eglinton 
Yonge-St.Clair 
York University Heights 
Yorkdale-Glen Park 
Lambton Baby Point 
Lansing-Westgate 
Lawrence Park North 
Lawrence Park South 
Leaside-Bennington 
Little Portugal 
Long Branch 
Malvern 
Maple Leaf 
Markland Wood 
Milliken 
Mimico 
Morningside 
Moss Park 
Mount Dennis 
Mount Olive-Silverstone-Jamestown 
Mount Pleasant East 
Mount Pleasant West 
New Toronto 
Newtonbrook East 
Newtonbrook West 
Niagara 
North Riverdale 
North St.James Town 
O'Connor-Parkview 
Oakridge 
Oakwood Village 
Old East York 
Palmerston-Little Italy 
Parkwoods-Donalda 
Pelmo Park-Humberlea 
Playter Estates-Danforth 
Pleasant View 
Princess-Rosethorn 
Regent Park 
Rexdale-Kipling 
Rockcliffe-Smythe 
Roncesvalles 
Rosedale-Moore Park 
Rouge 
Runnymede-Bloor West Village 
Rustic 
Scarborough Village 
South Parkdale 
South Riverdale 
St.Andrew-Windfields 
Steeles 
Stonegate-Queensway 
Tam O'Shanter-Sullivan 
Taylor-Massey 
The Beaches 
Thistletown-Beaumond Heights 
Thorncliffe P

In [52]:
print(Toronto_venues.shape)
Toronto_venues.head()

(1545, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wychwood,43.676919,-79.425515,Wychwood Barns Farmers' Market,43.680010,-79.423849,Farmers Market
1,Wychwood,43.676919,-79.425515,Wychwood Barns,43.680028,-79.423810,Event Space
2,Wychwood,43.676919,-79.425515,Hillcrest Park,43.676012,-79.424787,Park
3,Wychwood,43.676919,-79.425515,Wychwood Barns Community Gallery,43.679386,-79.424254,Art Gallery
4,Yonge-Eglinton,43.704689,-79.403590,North Toronto Memorial Community Centre,43.706098,-79.404337,Gym


In [53]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt North,19,19,19,19,19,19
Agincourt South-Malvern West,19,19,19,19,19,19
Alderwood,4,4,4,4,4,4
Annex,24,24,24,24,24,24
Banbury-Don Mills,22,22,22,22,22,22
Bathurst Manor,4,4,4,4,4,4
Bay Street Corridor,72,72,72,72,72,72
Bayview Village,3,3,3,3,3,3
Bayview Woods-Steeles,3,3,3,3,3,3


In [55]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 257 uniques categories.


In [58]:
# one hot encoding
toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
neighbourhood = toronto_onehot['Neighborhood']
toronto_onehot.drop(labels=['Neighborhood'], axis=1,inplace = True)
toronto_onehot.insert(0, 'Neighborhood', neighbourhood)

toronto_onehot.head()

,Neighborhood,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beer Bar,Beer Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Quad,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hotel,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Optical Shop,Other Great Outdoors,Other Repair Shop,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Lab,Photography Studio,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Rock Climbing Spot,Salad Place,Salon / Barbershop,Sandwich Place,School,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Social Club,South American Restaurant,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Swiss Restaurant,Syrian Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Tree,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Wychwood,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [59]:
toronto_onehot.shape

(1545, 257)

In [60]:
Toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beer Bar,Beer Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Quad,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hotel,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Optical Shop,Other Great Outdoors,Other Repair Shop,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Lab,Photography Studio,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Rock Climbing Spot,Salad Place,Salon / Barbershop,Sandwich Place,School,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Social Club,South American Restaurant,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Swiss Restaurant,Syrian Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Tree,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Agincourt North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,0.052632,0.000000,0.00,0.000000,0.000000,0.0000,0.052632,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.0000

In [61]:
Toronto_grouped.shape

(97, 257)

In [62]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt North ----
                  venue  freq
0    Chinese Restaurant  0.11
1                  Park  0.05
2              Pharmacy  0.05
3  Fast Food Restaurant  0.05
4        Clothing Store  0.05


----Agincourt South-Malvern West ----
                  venue  freq
0    Chinese Restaurant  0.37
1       Motorcycle Shop  0.05
2      Malay Restaurant  0.05
3         Shopping Mall  0.05
4  Cantonese Restaurant  0.05


----Alderwood ----
                     venue  freq
0              Pizza Place  0.50
1        Convenience Store  0.25
2              Coffee Shop  0.25
3  New American Restaurant  0.00
4             Optical Shop  0.00


----Annex ----
            venue  freq
0             Pub  0.08
1            Park  0.08
2     Coffee Shop  0.08
3  Sandwich Place  0.08
4            Café  0.08


----Banbury-Don Mills ----
                  venue  freq
0          Gourmet Shop  0.09
1           Pizza Place  0.09
2        Clothing Store  0.09
3  Cantonese Restaurant  0.05
4       Bubble T

                venue  freq
0         Pizza Place  0.11
1        Dessert Shop  0.11
2  Italian Restaurant  0.07
3         Coffee Shop  0.07
4      Sandwich Place  0.07


----Mount Pleasant West ----
                venue  freq
0         Coffee Shop  0.11
1  Italian Restaurant  0.09
2                Café  0.07
3         Pizza Place  0.06
4    Sushi Restaurant  0.06


----New Toronto ----
                venue  freq
0   Indian Restaurant  0.14
1         Supermarket  0.14
2  Italian Restaurant  0.14
3                Park  0.14
4                 Pub  0.14


----Newtonbrook East ----
                  venue  freq
0           Bridal Shop   0.5
1                  Café   0.5
2    African Restaurant   0.0
3  Other Great Outdoors   0.0
4       Motorcycle Shop   0.0


----Niagara ----
         venue  freq
0   Theme Park  0.07
1   Restaurant  0.07
2         Park  0.07
3    Racetrack  0.04
4  Cheese Shop  0.04


----North Riverdale ----
       venue  freq
0       Park  0.33
1       Café  0.33
2    

4             Motorcycle Shop   0.0


----Willowdale West ----
                        venue  freq
0           Mobile Phone Shop   0.5
1                        Park   0.5
2        Other Great Outdoors   0.0
3  Modern European Restaurant   0.0
4             Motorcycle Shop   0.0


----Willowridge-Martingrove-Richview ----
                venue  freq
0                 Gym   1.0
1  African Restaurant   0.0
2   Other Repair Shop   0.0
3     Motorcycle Shop   0.0
4       Movie Theater   0.0


----Woburn ----
                        venue  freq
0  Construction & Landscaping  0.17
1                      Bakery  0.17
2           Mobile Phone Shop  0.17
3           Fish & Chips Shop  0.17
4     South Indian Restaurant  0.17


----Woodbine Corridor ----
            venue  freq
0   Grocery Store   0.2
1  Discount Store   0.2
2  Baseball Field   0.2
3        Bus Stop   0.2
4            Café   0.2


----Woodbine-Lumsden ----
          venue  freq
0          Park  0.22
1  Skating Rink  0.22
2   Vide

In [63]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [66]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt North,Chinese Restaurant,Convenience Store,Japanese Restaurant,Pharmacy,Fried Chicken Joint,Frozen Yogurt Shop,Sandwich Place,Liquor Store,Beer Store,Discount Store
1,Agincourt South-Malvern West,Chinese Restaurant,Motorcycle Shop,Cantonese Restaurant,Noodle House,Bank,Seafood Restaurant,Malay Restaurant,Pool Hall,Restaurant,Mediterranean Restaurant
2,Alderwood,Pizza Place,Convenience Store,Coffee Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Dog Run,Farmers Market
3,Annex,Park,Coffee Shop,Pub,Sandwich Place,Café,Pizza Place,French Restaurant,Pharmacy,Restaurant,Donut Shop
4,Banbury-Don Mills,Pizza Place,Gourmet Shop,Clothing Store,Pet Store,Coffee Shop,Shopping Mall,Shoe Store,Spa,Medical Center,Bubble Tea Shop


In [70]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 0, 1, 1, 0, 1])

In [80]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = Toronto_DF
Toronto_merged.head()

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='AREA_NAME')

Toronto_merged.head() # check the last columns!

,AREA_LONG_CODE,AREA_NAME,LONGITUDE,LATITUDE,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,094,Wychwood,-79.425515,43.676919,0.0,Farmers Market,Park,Art Gallery,Event Space,Farm,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Zoo Exhibit
1,100,Yonge-Eglinton,-79.403590,43.704689,1.0,Coffee Shop,Fast Food Restaurant,Gym,Movie Theater,Restaurant,Buffet,Juice Bar,Poutine Place,Breakfast Spot,Supermarket
2,097,Yonge-St.Clair,-79.397871,43.687859,1.0,Coffee Shop,Italian Restaurant,Sushi Restaurant,Café,Sandwich Place,Gym,Bagel Shop,Pub,Thai Restaurant,Restaurant
3,027,York University Heights,-79.488883,43.765736,1.0,Japanese Restaurant,Furniture / Home Store,Caribbean Restaurant,Metro Station,Pizza Place,Miscellaneous Shop,Massage Studio,Fast Food Restaurant,Bar,Bank
4,031,Yorkdale-Glen Park,-79.457108,43.714672,1.0,Fast Food Restaurant,Restaurant,Fried Chicken Joint,Italian Restaurant,Bank,Coffee Shop,Greek Restaurant,Grocery Store,Furniture / Home Store,Athletics & Sports


In [81]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,AREA_NAME,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Wychwood,Farmers Market,Park,Art Gallery,Event Space,Farm,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Zoo Exhibit
5,Lambton Baby Point,River,Park,Garden,Zoo Exhibit,Event Space,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Dog Run
6,Lansing-Westgate,Health & Beauty Service,Park,Falafel Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Zoo Exhibit,Dive Bar
15,Milliken,Park,Sandwich Place,Mobile Phone Shop,Dive Bar,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
20,Mount Olive-Silverstone-Jamestown,Japanese Restaurant,Park,Coffee Shop,Dog Run,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Zoo Exhibit
27,North Riverdale,Café,Park,Pool,Dog Run,Zoo Exhibit,Event Space,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Farm
34,Parkwoods-Donalda,Food & Drink Shop,Pool,Park,Falafel Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Farm
43,Rosedale-Moore Park,Playground,Candy Store,Tennis Court,Park,Event Space,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Dog Run
54,Taylor-Massey,Baseball Field,Park,Theater,Coffee Shop,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Farm,Dog Run
62,West Hill,Park,Construction & Landscaping,Gym / Fitness Center,Event Space,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Dive Bar
